In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/Users/xiwencui/Desktop/Columbia/Fall 23/4575-RL/Project/data/yelp_dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 

# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/Users/xiwencui/Desktop/Columbia/Fall 23/4575-RL/Project/data/yelp_dataset/yelp_academic_dataset_checkin.json
/Users/xiwencui/Desktop/Columbia/Fall 23/4575-RL/Project/data/yelp_dataset/Dataset_User_Agreement.pdf
/Users/xiwencui/Desktop/Columbia/Fall 23/4575-RL/Project/data/yelp_dataset/yelp_academic_dataset_tip.json
/Users/xiwencui/Desktop/Columbia/Fall 23/4575-RL/Project/data/yelp_dataset/yelp_academic_dataset_review.json
/Users/xiwencui/Desktop/Columbia/Fall 23/4575-RL/Project/data/yelp_dataset/yelp_academic_dataset_business.json
/Users/xiwencui/Desktop/Columbia/Fall 23/4575-RL/Project/data/yelp_dataset/yelp_academic_dataset_user.json


In [2]:
# import the data (chunksize returns jsonReader for iteration)
businesses = pd.read_json("/Users/xiwencui/Desktop/Columbia/Fall 23/4575-RL/Project/data/yelp_dataset/yelp_academic_dataset_business.json", lines=True, orient='columns', chunksize=2000000)
reviews = pd.read_json("/Users/xiwencui/Desktop/Columbia/Fall 23/4575-RL/Project/data/yelp_dataset/yelp_academic_dataset_review.json", lines=True, orient='columns', chunksize=2000000)
users = pd.read_json("/Users/xiwencui/Desktop/Columbia/Fall 23/4575-RL/Project/data/yelp_dataset/yelp_academic_dataset_user.json", lines=True, orient='columns', chunksize=2000000)

In [3]:
# read data 
for business in businesses:
    subset_business = business[['business_id', 'name', 'address', 'city', 'state', 'postal_code', 'stars', 'review_count', 'is_open', 'attributes', 'categories']]
    break
    
for review in reviews:
    subset_review = review[['review_id', 'user_id', 'business_id', 'stars', 'date']]
    break

for user in users:
    subset_user = user[['user_id', 'name', 'review_count', 'yelping_since', 'average_stars']]
    break

In [4]:
print(subset_business.shape)
print(subset_review.shape)
print(subset_user.shape)
# peak the tables
display(subset_business.head(2))
display(subset_review.head(2))
display(subset_user.head(2))

(150346, 11)
(2000000, 5)
(1987897, 5)


,business_id,name,address,city,state,postal_code,stars,review_count,is_open,attributes,categories
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop..."
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma..."


,review_id,user_id,business_id,stars,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,2012-01-03 15:28:18


,user_id,name,review_count,yelping_since,average_stars
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,3.91
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,3.74


We only take restaurants that are currently open into consideration.

In [5]:
rest = subset_business[(subset_business['is_open'] == 1) & (subset_business['categories'].str.contains('Restaurant.*') == True)].reset_index()
rest = rest[['business_id','name','address', 'city', 'state', 'categories', 'attributes','stars']]
rest

,business_id,name,address,city,state,categories,attributes,stars
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'RestaurantsDelivery': 'False', 'OutdoorSeati...",4.0
1,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,"Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'BusinessParking': 'None', 'BusinessAcceptsCr...",2.0
2,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,"Ice Cream & Frozen Yogurt, Fast Food, Burgers,...","{'RestaurantsAttire': ''casual'', 'Restaurants...",1.5
3,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,FL,"Vietnamese, Food, Restaurants, Food Trucks","{'Alcohol': ''none'', 'OutdoorSeating': 'None'...",4.0
4,il_Ro8jwPlHresjw9EGmBg,Denny's,8901 US 31 S,Indianapolis,IN,"American (Traditional), Restaurants, Diners, B...","{'RestaurantsReservations': 'False', 'Restaura...",2.5
...,...,...,...,...,...,...,...,...
34999,w_4xUt-1AyY2ZwKtnjW0Xg,Bittercreek Alehouse,246 N 8th St,Boise,ID,"Bars, Gastropubs, Sandwiches, Nightlife, Resta...","{'BikeParking': 'True', 'Alcohol': 'u'full_bar...",4.5
35000,l9eLGG9ZKpLJzboZq-9LRQ,Wawa,19 N Bishop Ave,Clifton Heights,PA,"Restaurants, Sandwiches, Convenience Stores, C...","{'BikeParking': 'True', 'BusinessAcceptsCredit...",3.0
35001,cM6V90ExQD6KMSU3rRB5ZA,Dutch Bros Coffee,1181 N Milwaukee St,Boise,ID,"Cafes, Juice Bars & Smoothies, Coffee & Tea, R...","{'WiFi': ''free'', 'RestaurantsGoodForGroups':...",4.0
35002,WnT9NIzQgLlILjPT0kEcsQ,Adelita Taqueria & Restaurant,1108 S 9th St,Philadelphia,PA,"Restaurants, Mexican","{'WheelchairAccessible': 'False', 'Restaurants...",4.5
